In [1]:
import pandas as pd
import numpy as np

In [2]:
data_path = '..\\data\\'

In [3]:
target_data = pd.read_feather(data_path + 'interim\\correct_target.frt',
                              columns=['clnt_id', 'quarter', 'transactions_count', 'paid_avg_correct'])
target_data.sort_values(['clnt_id', 'quarter'], ascending=True, inplace=True)
target_data['quarter'] = target_data.quarter.astype(str)

internal_data = pd.read_feather(data_path + 'interim\\internal_features.frt')
internal_data['quarter'] = internal_data.quarter.astype(str)
internal_features = internal_data.columns[1:].values.tolist()

external_data = pd.read_feather(data_path + 'interim\\external_features.frt')
external_data['quarter'] = external_data.quarter.astype(str)

In [4]:
base = pd.merge(target_data, external_data, on=['clnt_id', 'quarter'], how='left')
base = pd.merge(base, internal_data, on='quarter', how='left')

In [5]:
shifted_internal_data = base[['clnt_id', 'quarter'] + internal_features].shift(-1)\
    .add_suffix('__prev').reset_index(drop=True)

In [6]:
base = pd.concat([base, shifted_internal_data], axis=1)
base[base.clnt_id != base.clnt_id__prev] = np.nan
base.dropna(inplace=True)
base.drop(columns=internal_features + ['clnt_id__prev', 'quarter__prev'], inplace=True)
base.reset_index(drop=True, inplace=True)

In [9]:
# base.to_feather(data_path + 'interim\\full_base.frt')

d:\Programs\Python\Python311\Lib\site-packages\pyarrow\pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):
